In [6]:
from time import time
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D, Activation, MaxPooling2D, Dense, Dropout, Flatten, AtrousConvolution2D
from spp.SpatialPyramidPooling import SpatialPyramidPooling
from PIL import Image
from keras import utils
from keras import optimizers
from keras.callbacks import TensorBoard
%matplotlib inline

In [3]:
batch_size = 64
num_channels = 3
num_classes = 10 

In [4]:
from keras.datasets import cifar10

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 106s 1us/step


In [7]:
print(x_train.shape)

(50000, 32, 32, 3)


In [8]:
def array_add_channel(a):
    '''
    Take an array of images and reshapes the 2D image to hae an added channel single array
    
    Parameters:
    -----------
    a : 3d numpy array; 
        first dimesion is the number of images
        second dimesion is the rows
        third dimension is the cols
        NOTE: tensorflow likes data to be in the order (Number, Height, Width, Channel)
    '''
    return(a.reshape(a.shape[0], a.shape[1], a.shape[2], 1))

def pre_process_x(a):
    '''
    Takes a raw array of images, add_channels the data, converts to float, and normalizes
    '''
    #a = array_add_channel(a)
    return(a.astype('float32')/255)

In [9]:
x_train = pre_process_x(x_train)
x_test = pre_process_x(x_test)
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [30]:
print(y_train.shape)
print(x_train.shape)

(50000, 10)
(50000, 32, 32, 3)


In [31]:
def simple_model(kernel, input_shape, num_classes):
    model = Sequential()
    model.add(Convolution2D(32, kernel, padding='same', activation='relu',input_shape=input_shape))
    model.add(Convolution2D(32, kernel, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    #model.add(Convolution2D(64, kernel, activation='relu', padding='same')) 
    #model.add(Convolution2D(64, kernel, activation='relu')) 
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.25))
    
    model.add(Convolution2D(128, kernel, dilation_rate=(2,2), activation='relu', border_mode='valid'))
    model.add(Convolution2D(128, kernel, dilation_rate=(2,2), activation='relu', border_mode='valid'))
    
    model.add(Flatten())
    model.add(Dense((32*32), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return(model)

In [32]:
kernel = (3,3)
input_shape = x_train.shape[1:]

In [34]:
m1 = simple_model(kernel, input_shape, num_classes)
m1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 11, 11, 128)       36992     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid", dilation_rate=(2, 2), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid", dilation_rate=(2, 2), activation="relu")`
  


In [35]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

In [36]:
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
m1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [38]:
m1.fit(x_train, y_train, batch_size=100, epochs=5, verbose=1, callbacks=[tensorboard])

Epoch 1/5
50000/50000 [==============================] - 13s 254us/step - loss: 0.2216 - acc: 0.9212
Epoch 2/5
50000/50000 [==============================] - 13s 254us/step - loss: 0.2063 - acc: 0.9286
Epoch 3/5
50000/50000 [==============================] - 13s 254us/step - loss: 0.1882 - acc: 0.9327
Epoch 4/5
50000/50000 [==============================] - 13s 255us/step - loss: 0.1825 - acc: 0.9363
Epoch 5/5
50000/50000 [==============================] - 13s 262us/step - loss: 0.1715 - acc: 0.9413
